# ***CS5040: Linear Optimization***

#### *Assignment-2*

Team Members:

1.   Aaditya Kehdekar- EE19BTECH11027
2.   Akshay Magre- EE19BTECH11002
3.   Megavath Praveen- AI19BTECH11018
4.   Tushar More- EE19BTECH11025





Question 2.

*   We are given a Matrix A of order m*n, vector c of length n, vector b of length m and some feasible point z of length n.
*   We have to take into consideration the following assumptions.
    * Polytope Ax<=b is non-degenerate.
    * Rank of A is n.

* First we will check if given polytope is bounded or unbounded.
* If it is bounded then We will maximize the given objective using proper algorithm for the above given conditions, such that Ax<=b is satisfied and the cost (c.T)x gets maximised. x is the solution of our optimization problem.

In [22]:
import numpy as np
from scipy.linalg import null_space

#step 1 : Moving to a vertex

In [26]:
# function to get tight and untight rows of matrix A
def tight_untight_rows(A,z,b):
  k = A@z
  At = []
  Au = []
  bt = []
  bu = []
  
  for i in range(len(b)) :
    
    if np.abs(k[i] - b[i])<0.00001:
      At.append(A[i])
      bt.append(b[i])
    else:
      Au.append(A[i])
      bu.append(b[i])
  
  return np.array(At),np.array(Au),np.array(bt),np.array(bu)

# Function to get non-zero vectors from null-space of A
def get_non_zero_vec_from_NS(A):
  ns = null_space(A)
  ns * np.sign(ns[0,0])

  if len(ns[0])==1:
    return ns.flatten()
  return ns[0].T
  
#step 1 : Moving to a vertex
def moving_to_vertex(A,b,c,z):
  ans = z
  # At point z divide rows of A into tight rows (At) and un-tight rows (Au).
  At,Au,bt,bu = tight_untight_rows(A,ans,b)
  # If the rank of tight rows (At) is n: then z is a vertex 
  # and we completed step 1.
  if np.linalg.matrix_rank(At) == len(z):
    ans = z
  else :
      # If the rank of tight rows (At) is less than n:
      while np.linalg.matrix_rank(At)<len(z):
        if len(At) == 0:
          ans = np.array([ b[0]/(A[0][i]*len(ans)) for i in range(len(ans)) ])
        else :
          # Take any non-zero vector u in the null space of tight rows (At).
          u = get_non_zero_vec_from_NS(At)
          ch = 0
          # Find some positive/negative real number α such that 
          # at point z+ αu some row from un-tight rows (Au) at z will be tight row.
          while ch<len(Au):
              if np.dot(Au[ch],u)!= 0:
                break
              ch += 1
          alpha = (bu[ch] - np.dot(Au[ch],ans))/(np.dot(Au[ch],u))
          # Update z with z+ αu. Go to point 1.
          ans  = ans + alpha*u 
        At,Au,bt,bu = tight_untight_rows(A,ans,b)
  
  return ans.astype(int)

### Vertex Matching

In [31]:
# Check ff cost vector is not non-negative liner combination of tight rows (At).
def check_if_optimal(At,c):
  coeff = np.linalg.pinv(At.T)@c
  for i in coeff:
    if i<0:
      return False
  return True

# Step 2: Vertex marching
def move_to_optimal_vertex(A,b,c,z):
  z = z
  # At point z divide rows of A into tight rows (At) and un-tight rows (Au).
  At,Au,bt,bu = tight_untight_rows(A,ans,b)
  # Find a column B[:,i] of B such that c.(-B[:,i]) is positive.
  while check_if_optimal(At,c)!=True:
    B = np.linalg.pinv(At)
    for i in range(len(B[0])):
      if np.dot(c,B[:,i]) <0:
        v = B[:,i]
    # Make some row from un-tight rows (Au) at z will be tight row.    
    ch = 0
    while ch<len(Au):
      if np.abs(np.dot(Au[ch],v)) > 0.0001 and (np.dot(Au[ch],ans) - bu[ch])/(np.dot(Au[ch],v))>0:
          break
      ch += 1
    # If some row from un-tight rows (Au) at z will be not be tight row
    # then Polytope is unbounded.
    if(ch ==len(Au)):
      return "Polytope is unbounded."
      
    alpha = (np.dot(Au[ch],ans) - bu[ch])/(np.dot(Au[ch],v))
    print(alpha)
    z = z - alpha*v # Update z with z- αv. Go to point 1.
    
    for i in range(len(ans)):
      if np.abs(ans[i])<0.0001:
        ans[i] = 0
    At,Au,bt,bu = tight_untight_rows(A,z,b)
  return ans

In [30]:
import numpy as np
from scipy.linalg import null_space

#inputs
A = np.array([[-1, -1], [-1, 0], [0,-1]])
b = np.array([-4, -1, -1 ])
c = np.array([5, 2])
z = np.array([5, 5]) #Can be taken any point in feasibility

 
print(move_to_optimal_vertex(A,b,c,moving_to_vertex(A,b,c,z)))

1.9999999999999996
Polytope is unbounded.
